## Example workbook for DIC <-> EBSD linking

In [3]:
# Load third-party packages
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform as tf

# Load quat, ebsd and hrdic packages from defdap package
from defdap.quat import Quat
import defdap.ebsd as ebsd
import defdap.hrdic as hrdic

#Set plot behaviour (grain selection has only been tested in osx display mode)
%matplotlib tk

### Load in and display DIC map

In [2]:
#Load in DIC map
DicFilePath = "example_data/"
DicMap = hrdic.Map(DicFilePath, "B00005.txt")

#Set crop
DicMap.setCrop(xMin=15, xMax=28, yMin=30, yMax=15)

Loaded DaVis 8.1.5 data (dimensions: 586 x 510 pixels, sub-window size: 12 x 12 pixels)


In [4]:
DicMap.setScale(micrometrePerPixel=0.02)

In [5]:
# Display max shear map
DicMap.plotMaxShear(plotColourBar=True, scaleBar=True, vmin=0, vmax=0.1)

### Print stats table

In [6]:
DicMap.printStatsTable(percentiles=['Min', 5, 'Mean', 95,'Max'], components = ['mss', 'f11', 'f22'])

DicMap (dimensions: 543 x 465 pixels, sub-window size: 12 x 12 pixels, number of points: 252495)

Component	Min	P5	Mean	P95	Max	
mss		0.01	0.39	1.95	4.39	12.74	
f11		-9.26	-0.34	2.13	5.15	16.30	
f22		-13.78	-3.77	-1.10	0.88	7.63	



### Effective shear strain histogram

In [7]:
histData = DicMap.crop(DicMap.max_shear).flatten()

fig, ax = plt.subplots(1)

counts, bins, bars = ax.hist(histData, bins=np.linspace(0,0.1,100), color='r', histtype='step')

ax.set_xlabel("Effective shear strain")
ax.set_ylabel("Frequency")

fig.show()

### Load in EBSD map then calculate local misorientation and display it

In [8]:
# Load in EBSD map
EbsdFilePath = "example_data/Map Data 2-DIC area"
EbsdMap = ebsd.Map(EbsdFilePath, "cubic")

# Load in slip system definitions (optional)
EbsdMap.loadSlipSystems("defdap/slip_systems/cubic.txt")

# Rotate the map 180 degrees if necessary
EbsdMap.eulerAngleArray = EbsdMap.eulerAngleArray[:, ::-1, ::-1]
EbsdMap.bandContrastArray = EbsdMap.bandContrastArray[::-1, ::-1]
EbsdMap.phaseArray = EbsdMap.phaseArray[::-1, ::-1]

# Create the internal array of quaternions for orientation calculations
EbsdMap.buildQuatArray()

# Find boundaries with given misorientation tolerance (in degrees)
EbsdMap.findBoundaries(boundDef = 6)

# Find grains, minGrainSize is the minimum number of pixels for a grain
EbsdMap.findGrains(minGrainSize = 10)

# Calculate grain local misorientation (Grain Reference Orientation Deviation (GROD) w.r.t grain mean orientations)
EbsdMap.calcGrainMisOri(calcAxis = True)

Loaded EBSD data (dimensions: 1006 x 996 pixels, step size: 0.1 um)


In [9]:
EbsdMap.plotEulerMap()

In [11]:
EbsdMap.plotMisOriMap(plotGBs=True, vmin=0, vmax=4)

### Plot IPF of all grains mean orientation

In [12]:
# Calculate grain mean orientations (but is already done here as part of calcGrainMisOri above)
# EbsdMap.calcGrainAvOris()

grainMeanOris = [grain.refOri for grain in EbsdMap]
refDir = np.array([1, 0, 0])

Quat.plotIPF(grainMeanOris, refDir, EbsdMap.crystalSym)

### Find a grain of interest (click around the map)

In [13]:
EbsdMap.locateGrainID()

Grain ID: 123
Grain ID: 446


### Plot misorientation and IPF for isolated grain

In [44]:
currEbsdGrain = EbsdMap[EbsdMap.currGrainId]

In [45]:
currEbsdGrain.plotMisOri()

In [46]:
currEbsdGrain.plotMisOri(component=4)

In [47]:
plt.figure()
currEbsdGrain.plotOriSpread(direction=refDir, c='b', s=1, alpha=0.2)
currEbsdGrain.plotRefOri(direction=refDir, c='k', s=100)

### Plot phase map

In [29]:
EbsdMap.plotPhaseMap()

## Link EBSD map to DIC map

### Define homologous points

In [ ]:
# Example of use of interactive tool coming soon...

In [30]:
# Set homologous points   (x, y)
DicMap.homogPoints = np.array((
    (303, 408), (120, 392), (275, 25)
))

EbsdMap.homogPoints = np.array((
    (536, 776), (236, 790), (466, 160)
))

### Display map with homologous points

In [33]:
EbsdMap.plotBoundaryMap()
plt.scatter(x=EbsdMap.homogPoints[:, 0], y=EbsdMap.homogPoints[:, 1], c='y', s=60)

In [34]:
DicMap.plotMaxShear()
plt.scatter(x=DicMap.homogPoints[:, 0], y=DicMap.homogPoints[:, 1], c='y', s=60)

### Link the EBSD and DIC map

In [35]:
DicMap.linkEbsdMap(EbsdMap)

### Plot max shear with boundaries

In [36]:
DicMap.plotMaxShear(plotGBs=True, plotColourBar=True, scaleBar=True, vmin=0, vmax=0.1)

### Detect grains in the DIC map

In [37]:
DicMap.findGrains(minGrainSize=10)

### Locate a grain of interest (click around)

In [38]:
DicMap.locateGrainID(displaySelected=True)
# displaySelected will show the selcted grain in a separate figure window

Grain ID: 297
Number of bands detected: 3
Grain ID: 286
Number of bands detected: 3
Grain ID: 303
Number of bands detected: 1
Grain ID: 263
Number of bands detected: 2
Grain ID: 78
Number of bands detected: 1
Grain ID: 53
Number of bands detected: 2


### Plot max shear for isolated grain

In [48]:
currDicGrain = DicMap[DicMap.currGrainId]

In [49]:
currDicGrain.plotMaxShear()

### Plot a histogram of it

In [53]:
plt.figure()

plt.hist(currDicGrain.maxShearList, bins=100, histtype='step');

plt.xlabel("Effective shear strain")
plt.ylabel("Frequency")

Text(0, 0.5, 'Frequency')

### Plot misorientation for isolated grain

In [55]:
ebsdGrainId = DicMap.ebsdGrainIds[DicMap.currGrainId]
currEbsdGrain = EbsdMap[ebsdGrainId]

In [56]:
currEbsdGrain.plotMisOri()

### Plot max shear for isolated grain with slip traces

In [58]:
currDicGrain.calcSlipTraces()
currDicGrain.plotMaxShear(plotSlipTraces=True)

### Plot boundaries without crop to see rotation

In [ ]:
plt.figure()

warpedBoundaries = tf.warp(-DicMap.ebsdMap.boundaries.astype(float), DicMap.ebsdTransform) > 0.1

warpedBoundaries = -warpedBoundaries.astype(int)

plt.imshow(warpedBoundaries, cmap="gray")

In [59]:
np.mean?

Signature: np.mean(a, axis=None, dtype=None, out=None, keepdims=<no value>)
Docstring:
Compute the arithmetic mean along the specified axis.

Returns the average of the array elements.  The average is taken over
the flattened array by default, otherwise over the specified axis.
`float64` intermediate and return values are used for integer inputs.

Parameters
----------
a : array_like
    Array containing numbers whose mean is desired. If `a` is not an
    array, a conversion is attempted.
axis : None or int or tuple of ints, optional
    Axis or axes along which the means are computed. The default is to
    compute the mean of the flattened array.

    .. versionadded:: 1.7.0

    If this is a tuple of ints, a mean is performed over multiple axes,
    instead of a single axis or all the axes as before.
dtype : data-type, optional
    Type to use in computing the mean.  For integer inputs, the default
    is `float64`; for floating point inputs, it is the same as the
    input dtype.
ou